# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Milvus

In [1]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://129.201.70.35:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

/tmp/ipykernel_65/1508769170.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  my_emb = OllamaEmbeddings(base_url='http://129.201.70.35:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")
/tmp/ipykernel_65/1508769170.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(


In [2]:
results = vectordb.similarity_search(query="什么是vmax的上网日志系统？", k=2)
results

[Document(metadata={'pk': 460219909630597135, 'producer': 'Apache FOP Version 2.6', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'total_pages': 29, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230523214533+08'00'", 'page': 13}, page_content='4\xa0功能本章包含如下主题：\uf06c上网日志保存\n10\n\uf06c上网日志查询\n10\n\uf06c上网日志批量导入查询\n11\n\uf06c日志管理\n11\n\uf06c账号管理\n11\n\uf06c角色管理\n12\n\uf06c资源监控\n12\n\uf06c告警管理\n12\n\uf06c省级网关对接\n12\n\uf06c拨测结果自动比对功能\n12\n\uf06cNAT日志入库功能\n13\n\uf06c北向接口\n13\n\uf06c上网日志历史查询\n13\n\uf06c云化上网日志XDR查询\n13\n以下介绍ZXVMAX-S的主要的功能。\n4.1\xa0上网日志保存\uf06c支持使用Gbase数据库或HDFS保存上网日志。\uf06c上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。\uf06c支持自动清理超过保存时间的上网日志。\n4.2\xa0上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：\uf06c时间范围+公网IP

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [34]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://129.201.70.35:11434', model='qwen3:8B', temperature=0.1)

my_llm.invoke("你好")

'<think>\n好的，用户发来“你好”，我需要友好回应。首先，保持自然的中文口语，避免机械感。可以加入表情符号增加亲切感，比如👋。然后询问用户的需求，比如“有什么我可以帮你的吗？”，这样既主动又不显得过于侵入。注意语气要轻松，符合日常对话。检查是否有拼写错误，确保语句通顺。最后，保持简洁，不要信息过载。这样用户会觉得被重视，也方便进一步交流。\n</think>\n\n👋 你好呀！有什么我可以帮你的吗？😊'

In [35]:
# from langchain_community.llms import VLLMOpenAI  # 注意类名为 VLLMOpenAI[3](@ref)
# my_llm = VLLMOpenAI(
#     openai_api_key="token-abc123",          # vLLM 无需鉴权，设为空字符串[3](@ref)
#     openai_api_base="http://129.201.70.35:9991/v1",  # 服务端地址
#     model_name="my_qwen3_8b",  # 需与部署的模型路径一致
#     # max_tokens=1024,                # 控制生成文本最大长度
#     temperature=0.6,               # 生成多样性参数
#     streaming=True                  # 支持流式输出（可选）
# )
# response = my_llm.invoke("你是谁？/no_think")
# response

## 3. 构建检索问答链

prompts

In [36]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question", "sources"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [37]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


## 4.检索问答链效果测试

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [38]:
question_1 = "什么是ZXVMAX-S的5GC业务？/no_think"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])
# print(result["source_documents"])

大模型+知识库后回答 question_1 的结果：
<think>

</think>

ZXVMAX-S的5GC业务是指ZXVMAX-S多维价值分析系统在5G核心网（5G Core Network, 5GC）场景中的应用。该系统面向用户的网络运维和运营分析，能够从网元、用户、终端、业务等多个维度，对业务使用过程中的质量和特征进行全方位的挖掘。它支持实时分析（如网络质量、数据业务质量等）和事后分析（如用户投诉、问题分析等），为移动通信网络的运维和运营提供全面支撑。

ZXVMAX-S系统在5GC业务中，能够帮助运营商优化网络建设、提升运维效率，并挖掘用户数据价值，从而为互联网企业、银行、广告商、政府等单位提供行业应用支撑。

谢谢你的提问！


In [39]:
question_2 = "周杰伦是谁？/no_think"
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
<think>

</think>

我是一个VMAX运维助手，专注于提供与VMAX系统相关的技术支持和信息。关于周杰伦的问题，我无法提供相关信息。谢谢你的提问！


### 4.2 无知识库大模型自己回答的效果

In [15]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.predict(prompt_template)

/tmp/ipykernel_65/46941173.py:5: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  my_llm.predict(prompt_template)


'<think>\n\n</think>\n\nZXVMAX-S 是华为推出的一款面向5G网络的虚拟化核心网（5GC）解决方案，其核心功能是提供5G网络切片、多接入边缘计算（MEC）、网络功能虚拟化（NFV）等能力，支持运营商构建灵活、高效、可扩展的5G核心网。\n\n### ZXVMAX-S 的5GC业务主要包括以下几个方面：\n\n1. **5G核心网（5GC）功能支持**：\n   - 提供5G核心网的关键功能模块，如：\n     - **AMF（Access and Mobility Management Function）**：负责接入和移动性管理。\n     - **SMF（Session Management Function）**：负责会话管理。\n     - **UPF（User Plane Function）**：负责用户面数据的转发和处理。\n     - **AUSF（Authentication Server Function）**：负责用户认证。\n     - **UDM（Unified Data Management）**：统一数据管理。\n     - **NRF（Network Repository Function）**：网络功能注册和发现。\n     - **NSSF（Network Slice Selection Function）**：网络切片选择。\n\n2. **网络切片（Network Slicing）**：\n   - 支持按需创建和管理不同的网络切片，满足不同行业（如工业、车联网、智慧城市等）对网络性能、安全、隔离等的不同需求。\n\n3. **多接入边缘计算（MEC）**：\n   - 支持将计算和存储能力下沉到网络边缘，降低时延，提升用户体验，适用于实时应用如AR/VR、自动驾驶等。\n\n4. **云原生架构**：\n   - 基于云原生技术构建，具备高可扩展性、高可用性、快速部署和灵活运维的能力，支持运营商快速响应业务变化。\n\n5. **开放接口与生态兼容**：\n   - 提供丰富的开放API接口，支持与第三方应用、业务平台、云平台等的集成，构建开放的5G生态。\n\n6. **支持多种接入方式**：\n   - 支持多种无线接入技术（如5G NR、4G LTE、Wi-Fi等）的统一接入和管

In [16]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.predict(prompt_template)

'<think>\n\n</think>\n\n“严威”是一个常见的中文名字，可能指代不同领域中的不同人物。以下是几种可能的解释：\n\n1. **历史人物**：  \n   在中国历史中，“严威”并非一个广为人知的历史人物，可能是指某个地方官员、士人或普通百姓，但缺乏明确的历史记载。\n\n2. **现代人物**：  \n   - **严威（演员）**：中国内地男演员，曾参演多部影视剧，如《新还珠格格》《巴啦啦小魔仙》等。  \n   - **严威（作家/学者）**：可能指某位学者或作家，但需结合具体领域进一步确认。  \n   - **严威（企业/公司名称）**：可能为某公司或品牌名称，需结合上下文判断。\n\n3. **其他可能性**：  \n   - **化名或笔名**：可能为某位作家、艺术家或网络创作者的笔名。  \n   - **虚构人物**：可能出现在小说、影视或游戏中，需具体作品信息。\n\n**建议**：若需更准确的答案，请提供更多背景信息（如领域、相关作品或上下文）。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [11]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

/tmp/ipykernel_1270322/2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
# 修改后的Prompt模板（添加chat_history变量）
template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
# 创建对话链
qa_chain = ConversationalRetrievalChain.from_llm(
        llm=my_llm,
        retriever=vectordb.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_PROMPT},
        chain_type="stuff"
    )
    
    # result = qa_chain({"question": question})
    # return result["answer"]
    

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [13]:
questions = [
    "什么是ZXVMAX-S的5GC业务？",
    "包含哪些功能", 
    "整理成表格"  
]

for question in questions:
    result = qa_chain({"question": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

回答：<think>
好的，用户问的是“什么是ZXVMAX-S的5GC业务？”，我需要根据提供的上下文来回答。首先，我得仔细看看相关上下文里有没有提到ZXVMAX-S的5GC业务相关内容。

在上下文中，有提到“ZXVMAX-S多维价值分析系统产品描述（5GC业务）图5-8会话跟踪组网逻辑图”，这说明ZXVMAX-S的5GC业务可能与产品描述和组网逻辑有关。另外，其他部分主要涉及告警处理、接口完整率和合规率的问题，可能和5GC业务的具体功能或应用场景相关。

不过，上下文中的信息比较零散，没有直接定义5GC业务是什么。可能需要结合已知的5GC知识来补充。5GC（5G Core）是5G网络的核心部分，负责控制面和用户面功能，支持网络切片、边缘计算等特性。ZXVMAX-S作为多维价值分析系统，可能在5GC业务中用于数据收集、分析，帮助运营商优化网络性能和业务策略。

但根据用户提供的上下文，重点可能在于ZXVMAX-S系统如何支持5GC业务，比如通过会话跟踪、接口监控等。因此，回答应结合产品描述和组网逻辑，说明ZXVMAX-S在5GC中的角色，如数据分析、性能监控等，同时提到相关告警处理机制确保服务质量和合规性。

需要注意的是，用户提供的上下文可能没有详细说明5GC业务的具体定义，所以回答需要基于现有信息，并适当扩展，但避免超出上下文范围。最后，确保回答结束时有“谢谢你的提问！”。
</think>

ZXVMAX-S的5GC业务是指基于5G核心网（5G Core）的多维价值分析系统，主要用于支持5G网络中的业务数据分析、性能监控及优化。该系统通过采集和分析5GC（5G Core Cloud）相关接口（如S11、SGS等）的数据，帮助运营商实现对网络切片、会话管理、服务指标等关键业务的精细化运营，从而提升网络效率和用户体验。其核心功能包括告警处理、填充率监控（如接口完整率、合规率）以及组网逻辑分析（如会话跟踪），确保5GC业务的稳定运行和资源优化。  

谢谢你的提问！


回答：<think>
好的，用户现在问的是ZXVMAX-S的5GC业务包含哪些具体功能。我需要根据之前的对话历史和上下文来回答。首先，回顾之前的对话，用户已经询问过ZXVMAX-S的5GC业务是什么，而我的回答提到了多维价值分析系统，涉及5GC的组网逻辑、告警处理、接口完整率和合规率监控等。

现